In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
split = 4

In [4]:
adata_pred_ood = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/sciplex/vae_mean_pooling/adata_ood_with_predictions_{split}.h5ad")
adata_pred_test = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/sciplex/vae_mean_pooling/adata_test_with_predictions_{split}.h5ad")

In [5]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_{split}.h5ad"

In [6]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [10]:
adata_ref_test = adata_test[~adata_test.obs["condition"].str.contains("Vehicle")].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[~adata_ood.obs["condition"].str.contains("Vehicle")].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [11]:
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="pred_reconstruction")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].layers["pred_reconstruction"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="pred_reconstruction")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].layers["pred_reconstruction"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [12]:
adata_ood

AnnData object with n_obs × n_vars = 144362 × 2001
    obs: 'cell_type', 'dose', 'dose_character', 'dose_pattern', 'g1s_score', 'g2m_score', 'pathway', 'pathway_level_1', 'pathway_level_2', 'product_dose', 'product_name', 'proliferation_index', 'replicate', 'size_factor', 'target', 'vehicle', 'perturbation', 'drug', 'cell_line', 'logdose', 'condition', 'n_genes', 'pubchem_name', 'pubchem_ID', 'smiles', 'control', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'ood_5', 'split'
    uns: 'cell_line_dict', 'ecfp_dict'
    obsm: 'X_pca', 'ecfp'

In [13]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [14]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


ValueError: Dict key mismatch; expected keys: ['A549_(+)-JQ1_10.0', 'A549_(+)-JQ1_100.0', 'A549_(+)-JQ1_1000.0', 'A549_(+)-JQ1_10000.0', 'A549_A-366_10.0', 'A549_A-366_100.0', 'A549_A-366_1000.0', 'A549_A-366_10000.0', 'A549_ABT-737_10.0', 'A549_ABT-737_100.0', 'A549_ABT-737_1000.0', 'A549_ABT-737_10000.0', 'A549_AC480_(BMS-599626)_10.0', 'A549_AC480_(BMS-599626)_100.0', 'A549_AC480_(BMS-599626)_1000.0', 'A549_AC480_(BMS-599626)_10000.0', 'A549_AG-14361_10.0', 'A549_AG-14361_100.0', 'A549_AG-14361_1000.0', 'A549_AG-14361_10000.0', 'A549_AG-490_(Tyrphostin_B42)_10.0', 'A549_AG-490_(Tyrphostin_B42)_100.0', 'A549_AG-490_(Tyrphostin_B42)_1000.0', 'A549_AG-490_(Tyrphostin_B42)_10000.0', 'A549_AICAR_(Acadesine)_10.0', 'A549_AICAR_(Acadesine)_100.0', 'A549_AICAR_(Acadesine)_1000.0', 'A549_AICAR_(Acadesine)_10000.0', 'A549_AMG-900_10.0', 'A549_AMG-900_100.0', 'A549_AMG-900_1000.0', 'A549_AMG-900_10000.0', 'A549_AR-42_10.0', 'A549_AR-42_100.0', 'A549_AR-42_1000.0', 'A549_AR-42_10000.0', 'A549_A

In [ ]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics_fast, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [ ]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [ ]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [ ]:
ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [ ]:
mean_ood_metrics_decoded

In [ ]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

In [ ]:
mean_ood_metrics_encoded

In [ ]:
deg_mean_ood_metrics

In [ ]:
deg_mean_test_metrics

In [ ]:
ood_metrics_encoded

In [ ]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/results/otfm/vae_mean_pooling"

In [ ]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
